In [1]:
from kafka import KafkaProducer
from time import sleep
import json

from datetime import datetime
from csv import reader
import pandas as pd

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],api_version=(2,0,0))

In [ ]:
import time
import shutil
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
finished="D:\IMEWS - Architect\Python Code\listener\finished"

class Watcher:
    DIRECTORY_TO_WATCH = "D:\IMEWS - Architect\Python Code\listener\input"

    def __init__(self):
        self.observer = Observer()

    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except:
            self.observer.stop()
            print("Error")

        self.observer.join()


class Handler(FileSystemEventHandler):

    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None

        elif event.event_type == 'created':
            
            # Take any action here when a file is first created.
            print("Received created event - %s." % event.src_path)
            filename=event.src_path.split("\\")[-1]
            print(filename)
            if filename.startswith("malware"):
                data= pd.read_csv(event.src_path)
                records=data.to_dict('records')
                for row in records:
                    print(row)
                    producer.send('malware',bytes(str(row), 'utf-8'))
  
            shutil.move(event.src_path,finished)

        elif event.event_type == 'modified':
            # Taken any action here when a file is modified.
            print("Received modified event - %s." % event.src_path)
if __name__ == '__main__':
    w = Watcher()
    w.run()